In [12]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import sys
import csv

import time_series
#import awarp.awarp as awarpy
#import awarp_wrapper.awarpWrapper as awarp

sys.path.append('/Volumes/GoogleDrive/Mon Drive/Python-helpers')
from save_obj import *

csv.field_size_limit(sys.maxsize) # To avoid max field limit on loading df_ts_enc

9223372036854775807

# Hashtags

Explore the hashtags dataset in order to find users which use the same hashtags. 

In [13]:
%%time
df_hashtag = pd.read_csv("data/df_hashtag.csv", encoding='utf-8', engine='python')

CPU times: user 50.2 s, sys: 22.6 s, total: 1min 12s
Wall time: 1min 33s


In [14]:
print(len(df_hashtag.hashtag.unique()))
print(len(df_hashtag.author_id.unique()))

243508
210417


There are 243'508 unique hashtags and 210'417 unique authors


In [22]:
"TeSt".lower()

'test'

In [25]:
hashtags = [x.lower() for x in df_hashtag.hashtag if x is not None and not isinstance(x, float)]

In [31]:
macron = [x for x in hashtags if 'macron' in x]

In [32]:
len(macron)

518724

In [35]:
pd.Series(hashtags).value_counts()[:50]

giletsjaunes           1085290
macron                  216210
giletjaune               96937
macrondemission          95502
giletjaunes              80988
granddebatnational       70903
gilletsjaunes            62255
france                   54239
17novembre               51379
paris                    51158
macrondestitution        50306
lrem                     46278
granddebat               43270
foulardsrouges           39517
pouvoirdachat            36679
giletsjaune              33076
bfmtv                    32401
gilestjaunes             31420
lagrandeexplication      31304
castaner                 29282
yellowvests              29013
granddébat               25657
giletsjaunesparis        25243
ric                      24979
8decembre                24870
macron20h                24594
politique                24407
castanerdemission        23678
1erdecembre              23608
lci                      20637
benalla                  18640
gj                       18478
17novemb

First we will try to find if some authors use exactly the same hashtags. If several authors use exactly the same set of hashtags, they may be related. 

In [5]:
# Not used 
# for each hashtag, get the list of users who used it and sort by count of users
#authors_for_hashtag = (df_hashtag.groupby('hashtag')['author_id']
#                                .apply(lambda x: list(x))
#                                .reset_index(name='authors_id'))
#authors_for_hashtag['len'] = authors_for_hashtag['authors_id'].str.len()
#authors_for_hashtag.sort_values(by='len', 
#                                ascending=False, 
#                                inplace=True)#.drop(columns='len')

In [6]:
# For each author, get the list of hashtags used by this author 
hashtags_for_author = (df_hashtag.groupby('author_id')['hashtag']
                                .apply(lambda x: set(x))
                                .reset_index(name='hashtag_set'))
# Add a column 'hashtag_set_count' representing the size of each hashtag set
hashtags_for_author['hashtag_set_count'] = hashtags_for_author['hashtag_set'].str.len()
# Sort by the size of hashtag set
hashtags_for_author.sort_values(by='hashtag_set_count', 
                                ascending=False, 
                                inplace=True)#.drop(columns='len')
# Compute the hash of each hashtag set to allow groupby
hashtags_for_author['hashtag_set_hash'] = hashtags_for_author.hashtag_set.apply(
                                                                lambda x: hash(str(x)))

In [7]:
print(len(hashtags_for_author.hashtag_set_hash.unique()))

139748


There are 139'811 unique hashtags set used, so there are some authors who use the same hashtag set.

In [8]:
# For each set of hashtags used, compute the list of authors who used it
authors_set = (hashtags_for_author.groupby('hashtag_set_hash')['author_id']
                                  .apply(lambda x: set(x))
                                  .reset_index(name='author_id_set'))
# Add a column 'author_id_set_count' representing the size of each author set
authors_set['author_id_set_count'] = authors_set['author_id_set'].str.len()
# Sort by the size of author set
authors_set.sort_values(by='author_id_set_count', ascending=False, inplace=True)
# Filter all hashtags used by one author only
authors_set = authors_set[authors_set.author_id_set_count>1]

In [9]:
# Merge to retrieve the set of hashtags corresponding to the hash. 
hash_to_hashtags = hashtags_for_author.drop_duplicates(subset=["hashtag_set_hash"])
hash_to_hashtags = hash_to_hashtags[["hashtag_set_hash", 
                                     "hashtag_set", 
                                     "hashtag_set_count"]]
authors_set = authors_set.merge(hash_to_hashtags, on="hashtag_set_hash", how="left")

In [10]:
authors_set.head()

,hashtag_set_hash,author_id_set,author_id_set_count,hashtag_set,hashtag_set_count
0,-8209175976447842787,"{106469654, 101561624, 7092852, 305234315, 154...",12393,{GiletsJaunes},1
1,-3012446724695049981,"{2835500698, 907278099372539905, 1861897135, 7...",3603,{GiletJaunes},1
2,-5335666704062341375,"{940998399209431041, 868885032890454016, 14092...",2851,{giletsjaunes},1
3,-2131775071562816053,"{1616666161, 19156886, 99378568, 2178485980, 1...",2538,{GilletsJaunes},1
4,-4310723279598145649,"{119864854, 1641437762, 2243915711, 9805128485...",1414,{LaGrandeExplication},1


In [11]:
authors_set_filtered = authors_set[authors_set["hashtag_set_count"]>2]

In [14]:
authors_set_filtered.head()

,hashtag_set_hash,author_id_set,author_id_set_count,hashtag_set,hashtag_set_count
252,8106149020491007060,"{2803588417, 822349694198235136, 8705729432727...",28,"{GiletsJaunes, giletsjaunes, Giletsjaunes}",3
253,-1955107465152431919,"{961127978, 1096448516862500865, 9902736305747...",28,"{PasDeDebatSansUDI, GrandDebatNational, Europe...",3
270,7998818677806465741,"{1065630104976539649, 701784429618978818, 1294...",27,"{GiletsJaunes, GiletJaunes, giletsjaunes}",3
281,3764170619451331643,"{1072860802820923397, 2872898329, 2481858846, ...",26,"{GiletsJaunes, GiletJaune, GiletJaunes}",3
284,2715095747009216233,"{1061720214910443521, 1094204609093083137, 894...",26,"{JusticeFiscale, LAutreQuestionnaire, GiletsJa...",4


In [44]:
test = authors_set[authors_set["hashtag_set_count"]>5]

In [45]:
test.loc[528].author_id_set

{'1034645744907677696',
 '124854027',
 '1495487005',
 '170763654',
 '20699718',
 '2267764975',
 '27980991',
 '282919637',
 '383125195',
 '828226984815652865',
 '830999894',
 '91354290',
 '976225807',
 '987234589730136064'}

In [18]:
len(set([y for x in authors_set_filtered["author_id_set"] for y in x]))

2827

In [17]:
candidates = list(authors_set_filtered["author_id_set"].apply(list))
save_obj(candidates, "candidates/", "candidates_hashtag")

For each pair of authors in each author set, we will use data time warping to measure the similarity between them. We need first to extract the activity of each user. We use view_user_activity, which contains in each line the author and timestamp of a tweet. 

In [26]:
df_ts = None
if os.path.isfile("data/df_ts_enc.csv"):
    df_ts = pd.read_csv("data/df_ts_enc.csv", encoding='utf-8', engine='python')
else:
    df_activity = pd.read_csv("data/merged_tables/view_user_activity.csv", 
                                   encoding='utf-8', 
                                   engine='python')
    # Transform the created_at column into datetime
    df_activity["created_at_dt"] = df_activity["created_at"].apply(lambda x: 
                                        datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
    TS = time_series.TimeSeries()
    # compute time series for each author
    df_ts = (df_activity.groupby('author_id')['created_at_dt']
                .apply(lambda x: TS.get_encoded_ts(list(x)))
                .reset_index(name='ts'))
    df_ts.to_csv("data/df_ts_enc.csv", index=False, encoding='utf-8')

In [27]:
df_ts["ts"] = df_ts["ts"].map(lambda x: [int(y) for y in x[1:-1].split(",")])
df_ts["tweet_count"] = df_ts["ts"].apply(lambda x: sum([y for y in x if y>0]))
df_ts_5 = df_ts[df_ts.tweet_count > 4]
df_ts_5.author_id = df_ts_5.author_id.astype(str)

/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:5209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [28]:
all_authors = set(df_ts_5.author_id.unique())

In [13]:
# We could filter out all authors from user_activity that are not in df_hashtag
#unique_authors = df_hashtag.author_id.unique()
#df_activity = df_user_activity[df_user_activity.author_id.isin(unique_authors)]

In [30]:
# TEST
# compute for one group
batch = list(test.iloc[0].author_id_set)
print(len(batch))
batch = [x for x in batch if x in all_authors]
print(len(batch))

14
2


In [30]:
# TEST
# compute for one group

batch = authors_set.iloc[4].author_id_set
print(len(batch))
batch = [x for x in batch if x in all_authors]
print(len(batch))

1414
54


In [31]:
test = df_ts_5[df_ts_5.author_id.isin(batch)]

In [32]:
test

,author_id,ts,tweet_count
331958,383125195,"[-1, -3637765, 1, -2644641, 1, -84882, 1, -708...",67
1010071,828226984815652865,"[-1, -69132, 1, -8567, 1, -240001, 1, -184752,...",29


In [34]:
author_1 = df_hashtag[df_hashtag["author_id"]=="383125195"]
author_2 = df_hashtag[df_hashtag["author_id"]=="828226984815652865"]

In [36]:
for x in author_1.body:
    print(x)

Live - Multiplex - Gilets Jaunes - 17 Février #gilets #jaunes #direct #paris #yellowvest #live

https://t.co/domsuyQXGA
Live - Multiplex - Gilets Jaunes - 17 Février #gilets #jaunes #direct #paris #yellowvest #live

https://t.co/domsuyQXGA
Live - Multiplex - Gilets Jaunes - 17 Février #gilets #jaunes #direct #paris #yellowvest #live

https://t.co/domsuyQXGA
Live - Multiplex - Gilets Jaunes - 17 Février #gilets #jaunes #direct #paris #yellowvest #live

https://t.co/domsuyQXGA
Live - Multiplex - Gilets Jaunes - 17 Février #gilets #jaunes #direct #paris #yellowvest #live

https://t.co/domsuyQXGA
Live - Multiplex - Gilets Jaunes - 17 Février #gilets #jaunes #direct #paris #yellowvest #live

https://t.co/domsuyQXGA


In [37]:
for x in author_2.body:
    print(x)

J'aime une vidéo @YouTube : "Live - Multiplex - Gilets Jaunes Acte XI - France #gilets #jaunes #direct #paris #yellowvest #live" à
J'aime une vidéo @YouTube : "Live - Multiplex - Gilets Jaunes Acte XI - France #gilets #jaunes #direct #paris #yellowvest #live" à
J'aime une vidéo @YouTube : "Live - Multiplex - Gilets Jaunes Acte XI - France #gilets #jaunes #direct #paris #yellowvest #live" à
J'aime une vidéo @YouTube : "Live - Multiplex - Gilets Jaunes Acte XI - France #gilets #jaunes #direct #paris #yellowvest #live" à
J'aime une vidéo @YouTube : "Live - Multiplex - Gilets Jaunes Acte XI - France #gilets #jaunes #direct #paris #yellowvest #live" à
J'aime une vidéo @YouTube : "Live - Multiplex - Gilets Jaunes Acte XI - France #gilets #jaunes #direct #paris #yellowvest #live" à


In [69]:
current_idx = 0
author_list = list(test.author_id)
cur_count = test.shape[0]-1
count = int((test.shape[0]*(test.shape[0]-1))/2)
author_left = []
author_right = []
for i in range(len(author_list)-1):
    author_left += [author_list[i]]*cur_count
    cur_count-=1
cur_count = test.shape[0]-1
for i in range(len(author_list)-1):
    for j in range(i+1, len(author_list)):
        author_right.append(author_list[j])

print(len(author_left))
print(len(author_right))
        
for i in range(len(author_left)):
    print(author_left[i] + "  " + author_right[i])

    

10
10
27804286  104135815
27804286  127558901
27804286  142460588
27804286  144989956
104135815  127558901
104135815  142460588
104135815  144989956
127558901  142460588
127558901  144989956
142460588  144989956


In [71]:
def compute_awarp(df_author_ts):
    '''
    Takes a dataframe with 'author_id' and 'ts' as columnn. 
    The ts column is already encoded for awarp. 
    Write the time series on disk and call a python wrapper 
    for cpp that will read the file and compute the distances. 
    The wrapper will then write the result on disk. The function 
    then read the file and returns a dataframe with 'author_id_x', 
    'author_id_y' and 'd' representing the distance between two 
    authors.
    '''
    to_write = ""
    for x in df_author_ts.ts:
        for y in x:
            to_write += str(y) + ","
        to_write = to_write[:-1]
        to_write += "\n"

    with open("awarp_wrapper/ts.csv", "w") as f:
        f.write(to_write)
    
    # call the wrapper
    f = awarp.DTW()
    f.run()
    
    # read the output file
    df_out = pd.read_csv('awarp_wrapper/out.txt', sep=" ", header=None, 
                     names = ["author_id_x","author_id_y","d"])
    
    # update the author columns with the right authors
    current_idx = 0
    author_list = list(df_author_ts.author_id)
    cur_count = df_author_ts.shape[0]-1
    count = int((df_author_ts.shape[0]*(df_author_ts.shape[0]-1))/2)
    author_left = []
    author_right = []
    for i in range(len(author_list)-1):
        author_left += [author_list[i]]*cur_count
        cur_count-=1
    cur_count = df_author_ts.shape[0]-1
    for i in range(len(author_list)-1):
        for j in range(i+1, len(author_list)):
            author_right.append(author_list[j])
            
    df_out["author_id_x"] = author_left
    df_out["author_id_y"] = author_right
    
    return df_out
    

In [72]:
test_out = compute_awarp(test)
test_out.shape

(10, 3)

In [76]:
f = awarp.DTW()
f.run()

In [74]:
test.head()

,author_id,ts,tweet_count
57267,27804286,"[-1, -69006, 1, -38764, 1, -983, 1, -278, 1, -...",457
147746,104135815,"[-1, -3883849, 1, -291206, 1, -4186810, 1, -12...",6
170793,127558901,"[-1, -70609, 1, -177790, 1, -469171, 1, -47174...",94
182437,142460588,"[-1, -239227, 1, -644102, 1, -59558, 1, -97874...",6
184306,144989956,"[-1, -195650, 1, -458187, 1, -622285, 1, -1006...",9


In [73]:
test_out

,author_id_x,author_id_y,d
0,27804286,104135815,21.23680
1,27804286,127558901,19.05260
2,27804286,142460588,21.23680
3,27804286,144989956,21.16600
4,104135815,127558901,9.38083
5,104135815,142460588,0.00000
6,104135815,144989956,1.73205
7,127558901,142460588,9.38083
8,127558901,144989956,9.21954
9,142460588,144989956,1.73205


In [35]:
# The python version of awarp is much slower
#%%time
#distances = []
#for i in range(test.shape[0]):
#    for j in range(i+1, test.shape[0]):
#        d = awarpy.dtw_G(test.iloc[i].ts, test.iloc[j].ts)
#        distances.append(d)

In [ ]:
%%time
distances = []
for i in range(test.shape[0]):
    for j in range(i+1, test.shape[0]):
#        d = awarpy.dtw_G(test.iloc[i].ts, test.iloc[j].ts)
#        distances.append(d)

In [45]:
%%time
f = awarp.DTW()
f.run()

CPU times: user 717 µs, sys: 1.23 ms, total: 1.95 ms
Wall time: 7.54 ms


In [143]:
user_activity = df_user_activity[df_user_activity.author_id==test].copy()
if len(user_activity) > 4:
    print("Ok")

In [96]:
user_activity = df_user_activity[df_user_activity.author_id==test].copy()
ts = TS.get_ts_from_datetime(list(user_activity["created_at_dt"]))

NameError: name 'ts_size' is not defined

In [247]:
importlib.reload(dtw)

<module 'dtw_wrapper.dtwWrapper' from '/Volumes/GoogleDrive/Mon Drive/EPFL/__Master_5/SemesterProject/Exposing_french_botnet/dtw_wrapper/dtwWrapper.py'>

In [131]:
import importlib

In [140]:
importlib.reload(time_series)
TS = time_series.TimeSeries()

In [88]:
ts_list = []
for x in batch:
    print("test")
    user_activity = df_user_activity[df_user_activity.author_id==x].copy()
    ts = TS.get_ts_from_datetime(list(user_activity["created_at_dt"]))
    ts_list.append(ts)

test


NameError: name 'ts_size' is not defined